In [185]:
import folium
import pandas as pd

In [186]:
df = pd.read_pickle('data/pickled/cleaned_df.pkl')
df.reset_index(inplace=True)

In [187]:
df.head(2)

,Date Occurred,DR Number,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,Victim Age,Victim Sex,Victim Descent,Premise Code,Premise Description,Status Code,Status Description,latitude,longitude
0,2010-01-01,102120693,1117,21,Topanga,2156,354,THEFT OF IDENTITY,21,M,H,404.0,DEPARTMENT STORE,IC,Invest Cont,34.1875,-118.603
1,2010-01-01,100504041,2130,5,Harbor,563,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),0,X,X,104.0,DRIVEWAY,IC,Invest Cont,33.737,-118.2967


In [188]:
def generateBaseMap(default_location=[34.052235, -118.243683], default_zoom_start=9):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

In [189]:
# basic map of Los Angeles
empty_map = generateBaseMap()
empty_map

## Heat Map

This successfully generates, but it takes a few minutes to run.

In [190]:
# from folium import plugins
# from folium.plugins import HeatMap

# first_map = folium.Map(location=[34.052235, -118.243683],
#                     zoom_start = 9) 

# # Making sure that the coordinates are flaots
# df['latitude'] = df['latitude'].astype(float)
# df['longitude'] = df['longitude'].astype(float)

# # List comprehension to make out list of lists
# heat_data = [[row['latitude'],row['longitude']] for index, row in df.iterrows()]

# # Plot it on the map
# HeatMap(heat_data).add_to(first_map)

# # Display the map
# first_map

## Choropleth Map with LAPD Prescinct
Number of total reported crimes in each area of LA from 2010-2019

In [245]:
# there are 21 unqiue area IDs in LA
df['Area Name'].unique()

array(['Topanga', 'Harbor', 'Rampart', 'Olympic', 'Devonshire',
       'Southeast', 'Wilshire', 'Van Nuys', 'Northeast', 'West Valley',
       'Newton', 'Hollywood', 'Southwest', 'Foothill', 'Mission',
       '77th Street', 'Pacific', 'Central', 'West LA', 'Hollenbeck',
       'N Hollywood'], dtype=object)

In [193]:
# creating crime_df with each unique crime and associated area name
crime_df = df[['DR Number','Area Name']]
# renaming columns
crime_df.rename(columns={"DR Number": "IncidentID", 'Area Name':'Area'}, inplace=True)

In [249]:
crime_counts_df.sort_values(by='num_crimes', ascending=False)

,Area,num_crimes
0,77th Street,137513
15,Southwest,128111
8,North Hollywood,107707
12,Pacific,105655
14,Southeast,105104
7,Mission,98395
10,Northeast,94912
17,Van Nuys,94358
9,Newton,94123
6,Hollywood,92742


In [196]:
# fixing area names so they appear on the map
crime_counts_df['Area'] = crime_counts_df['Area'].str.replace('N Hollywood', 'North Hollywood')
crime_counts_df['Area'] = crime_counts_df['Area'].str.replace('West LA', 'West Los Angeles')

In [197]:
crime_counts_df.head()

,Area,num_crimes
0,77th Street,137513
1,Central,90489
2,Devonshire,91347
3,Foothill,75348
4,Harbor,86972


In [195]:
# creating crime_counts_df with the count of crime in each area
crime_counts_df = crime_df.groupby('Area').size().reset_index()
crime_counts_df = crime_counts_df.rename({0: 'num_crimes'}, axis='columns')

In [241]:
LAgeo = r'data/LAPD_Divisions.json'

mapLACrimes = folium.Map(
    location=[34.052235, -118.243683], 
    zoom_start=10, 
    tiles='cartodbpositron', 
)

mapLACrimes.choropleth(
    geo_data=LAgeo,
    name='choropleth',
    data=crime_counts_df,
    columns=['Area', 'num_crimes'],
    key_on='feature.properties.name',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Reported Crimes in LA from 2010-2019'
)

In [242]:
mapLACrimes